<a href="https://colab.research.google.com/github/WenjunYuAnny/Artificial_Intelligence_Assignment/blob/main/Assignment_3/Assignment3_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
ROOT_DIR = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/txt_dataset'

In [3]:
import cv2

In [4]:
import csv

In [5]:
!pip install ultralytics

In [6]:
import os
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

#train the model running 100 epochs
I also tried using xl model, but it doesn't improve the result significantly so here I just stick to the fastest model and try to compensate by running more epochs

In [ ]:
results = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), epochs=100)

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/txt_dataset/config.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=Fals

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#save the model
!scp -r /content/runs '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/txt_dataset'

In [7]:
model_path = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/txt_dataset/runs/detect/train/weights/best.pt'

In [8]:
trained_model = YOLO(model_path)
threshold = 0.5

#Process the video directory to get drone detection frames and videos

In [10]:
video_dir = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/video'
output_video_dir = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/video_predicted'
annotated_images_dir = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/annotated_images'
original_images_dir = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/detected_images'
output_bbox_dir = '/content/gdrive/MyDrive/2023 Fall Artificial Intelligence/Drone_detection/bbox'

os.makedirs(output_video_dir, exist_ok=True)
os.makedirs(annotated_images_dir, exist_ok=True)
os.makedirs(original_images_dir, exist_ok=True)
os.makedirs(output_bbox_dir, exist_ok=True)

for filename in os.listdir(video_dir):
    if filename.lower().endswith('.mp4'):
        video_path = os.path.join(video_dir, filename)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Failed to open video: {filename}")
            continue
        else:
            print(f"Processing {filename}")

        original_frame_rate = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(os.path.join(output_video_dir, filename), fourcc, original_frame_rate, (int(cap.get(3)), int(cap.get(4))))

        bbox_info = []
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = trained_model(frame)[0]
            for result in results.boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = result
                if score > threshold:
                    #save the frame that has a drone detected
                    cv2.imwrite(os.path.join(original_images_dir, f'{filename}_frame{frame_count}.jpg'), frame)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
                    cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
                    #save bounding box
                    bbox_info.append([frame_count, x1, y1, x2, y2, score, results.names[int(class_id)]])
                    #save the frame with the bouding box annotation
                    cv2.imwrite(os.path.join(annotated_images_dir, f'{filename}_frame{frame_count}.jpg'), frame)

            out.write(frame)
            frame_count += 1

        cap.release()
        out.release()

        #save the bouding box info to a csv
        bbox_file_path = os.path.join(output_bbox_dir, f'{filename}.csv')
        with open(bbox_file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['frame', 'x1', 'y1', 'x2', 'y2', 'score', 'class'])
            writer.writerows(bbox_info)



0: 384x640 1 drone, 6.6ms
Speed: 2.8ms preprocess, 6.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 6.1ms
Speed: 1.9ms preprocess, 6.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 6.3ms
Speed: 1.6ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing Drone Tracking 1.mp4


Streaming output truncated to the last 5000 lines.
Speed: 1.9ms preprocess, 7.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 7.4ms
Speed: 1.9ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 drones, 7.1ms
Speed: 1.8ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 drones, 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 drones, 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 drones, 6.8ms
Speed: 1.8ms preprocess, 6.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 7.3ms
Speed: 1.6ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 drone, 7.3ms
Speed: 1.8ms preprocess, 7.3ms inference, 1.4ms postprocess per image at shape (1, 3

Processing Drone tracking 2.mp4


Streaming output truncated to the last 5000 lines.
0: 384x640 (no detections), 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.8ms
Speed: 1.8ms preprocess, 6.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.8ms
Speed: 1.6ms preprocess, 7.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.8ms
Speed: 1.7ms preprocess, 7.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.8ms
Speed: 1.6ms preprocess, 6.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.7m

### Screenshots of frames with prediction and video with prediction are in the same folder on Github.